### Preparations
---

Define seed

In [ ]:
seed = 42

Clone the repository

In [ ]:
!git clone https://github.com/Godofnothing/CLIP_experimental

Installation of the dependencies

In [ ]:
!pip install -q pytorch-lightning
!pip install -q ftfy
!pip install efficientnet_pytorch
!pip install torchvision

In [ ]:
import sys
sys.path.insert(0, "CLIP_experimental")

import torch
import numpy as np

from src import ImageLabelDataset, train_val_split

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
assert device == 'cuda'

In [ ]:
backbone = 'resnet' # either 'resnet' or 'efficientnet'
assert backbone in ['resnet', 'efficientnet']
freeze_backbone = True

### Get dataset
---

In [ ]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download moltean/fruits
!unzip -q fruits.zip
dataset_root = 'fruits-360'

kaggle.json
 99% 751M/760M [00:05<00:00, 133MB/s]
100% 760M/760M [00:05<00:00, 135MB/s]


### Construction of the dataset and loaders
---

In [ ]:
import torchvision.transforms as T
from torch.utils.data import DataLoader

In [ ]:
MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]

train_transform = T.Compose([
        T.Resize(224, interpolation=T.InterpolationMode.BICUBIC),
        # augmentation
        T.RandomHorizontalFlip(p=0.5),
        T.RandomPerspective(),
        T.RandomRotation(degrees=20, interpolation=T.InterpolationMode.BICUBIC),
        T.GaussianBlur(3, sigma=(0.1, 2.0)),
        #
        T.ToTensor(),
        T.Normalize(MEAN, STD)                              
])

val_transform = T.Compose([
    T.Resize(224, interpolation=T.InterpolationMode.BICUBIC),
    T.ToTensor(),
    T.Normalize(MEAN, STD)                              
])

In [ ]:
train_val_dataset = ImageLabelDataset('Fruit-Images-Dataset/Training', image_transform=train_transform)
train_dataset, val_dataset = train_val_split(train_val_dataset)
val_dataset.image_transform = val_transform

test_dataset = ImageLabelDataset('Fruit-Images-Dataset/Test', image_transform=val_transform)
 
if freeze_backbone:
    batch_size = 48
elif backbone == 'efficientnet':
    batch_size = 16
else:
    batch_size = 32

In [ ]:
import os

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=os.cpu_count())
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=os.cpu_count())
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=os.cpu_count())

### The training procedure
---

Init Pytorch Lightning modules

In [ ]:
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.utilities.seed import seed_everything

from efficientnet_pytorch import EfficientNet
from torchvision import models

seed_everything(seed, workers=True)

In [ ]:
if backbone == 'efficientnet':
  effnet = EfficientNet.from_pretrained("efficientnet-b7")
  effnet._dropout = nn.Identity()
  effnet._fc = nn.Identity()
  n_features = 2560
  model = ImageClassifier(effnet, n_features, 131, init_learning_rate=8e-3, freeze_backbone=freeze_backbone)
elif backbone == 'resnet':
  resnet = models.resnet101(pretrained=True)
  resnet.fc = nn.Identity()
  n_features = 2048
  model = ImageClassifier(resnet, n_features, 131, freeze_backbone=freeze_backbone)

Loaded pretrained weights for efficientnet-b7


In [ ]:
log_dir = f'logs/CNN_backbone={backbone}_freeze_backbone={freeze_backbone}'
logger = TensorBoardLogger(log_dir)
checkpoint = ModelCheckpoint(log_dir, monitor='val/accuracy', mode='max')

if backbone == 'efficientnet' and not freeze_backbone:
    accumulate_grad_batches = 2
else:
    accumulate_grad_batches = 1
    
max_epochs = 5 if not freeze_backbone else 10

trainer = pl.Trainer(
    gpus=1,
    max_epochs=max_epochs,
    deterministic=True,
    amp_backend='native',
    auto_lr_find=True,
    logger=logger,
    callbacks=[checkpoint],
    val_check_interval=0.5,
    accumulate_grad_batches=accumulate_grad_batches
)

if backbone != 'efficientnet' or not freeze_backbone:
    trainer.tune(model, train_dataloader=train_loader)
trainer.fit(model, train_loader, val_loader)
model.load_state_dict(torch.load(checkpoint.best_model_path)['state_dict'])
trainer.test(model, test_loader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type         | Params
--------------------------------------------
0 | backbone   | EfficientNet | 63.8 M
1 | classifier | Linear       | 335 K 
--------------------------------------------
335 K     Trainable params
63.8 M    Non-trainable params
64.1 M    Total params
256.490   Total estimated model params size (MB)


Global seed set to 42


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test/accuracy': 0.9462270736694336}
--------------------------------------------------------------------------------


[{'test/accuracy': 0.9462270736694336}]

In [ ]:
Tensorboard logger
---

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs